## Suites: Banking
#### Web url:   
http://211.25.204.108:30002/

Using Library Python Selenium to test the web site

In [96]:
# ####### DEBUG ONLY #######
# ####### Load the "autoreload" extension #######
%load_ext autoreload
# ####### always reload modules marked with "%import" #######
%autoreload 2

import os
import sys

sys.path.append("../config")
sys.path.append("../util")
sys.path.append("../tests")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [97]:
# Importing packages
from config import config
from datetime import datetime, timedelta
import json
import logging
import logging.config
import pathlib
from pprint import pprint
import fx_test
from fx_test import TestEngine

from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.common.keys import Keys

import unittest
from ui_helper import UIHelper

## Variables Declaration

In [98]:
log_config = "../config/logging.conf"


logging.config.fileConfig(log_config, disable_existing_loggers=False)
logger = logging.getLogger(__name__)

suite_name = "suite_banking"
suite_case = "suite_02"
logger.info("Test Suite: {}".format(suite_name))
logger.info("Suite Case: {}".format(suite_case))

[INFO] Test Suite: suite_banking
[INFO] Suite Case: suite_02


### Main Function Begin
def main(args):

In [99]:
mode = config["web"]["mode"]
base_url = config["web"]["base_url"]
timeout = config["web"]["timeout"]
user_name = config["web"]["user_name"]
password = config["web"]["password"]

if mode == "windows":
    driver_path = "../../../library/drivers/chrome/win/chromedriver.exe"
else:
    driver_path = "../../../library/drivers/chrome/mac/chromedriver"
    
logger.info("Mode: {}".format(mode))
logger.info("Base Url: {}".format(base_url))
logger.info("User Name: {}".format(user_name))

[INFO] Mode: windows
[INFO] Base Url: http://211.25.204.108:30002/
[INFO] User Name: zong.xian.soh@adv-fusionex.com


In [100]:
# Get web page
instance = TestEngine(mode="windows", driver_path="../../../library/drivers/chrome/win/chromedriver.exe", timeout=30)
instance.get_driver().get(base_url)

WebDriver created!
Mode: windows
Driver Path: ../../../library/drivers/chrome/win/chromedriver.exe
Timeout: 10


<a id='top'></a>
## Table of contents
- <a href='#data_load'>Load Test Data</a>
- <a href='#login'>Login</a>
- <a href='#switch_company'>Switch Company</a>
- <a href='#banking_account'>Banking - Account</a>
- <a href='#banking_transfer'>Banking - Transfer</a>
- <a href='#banking_transaction'>Banking - Transaction</a>





In [101]:
##Launch uihelper
ui = UIHelper(instance, user_name=user_name, password=password)

<a id='data_load'></a>
### Load Test Data
<a href='#top'>top</a>

In [102]:
# Load sample data
data_path =  f"../data/{suite_case}/banking.json"
data = ui.load_data(data_path)

transfers = data["transfers"]
    

<a id='login'></a>
### Login
<a href='#top'>top</a>

In [103]:
ui.login()

<a id='switch_company'></a>
### Switch Company
<a href='#top'>top</a>

In [ ]:
ui.switch_company(data["company"]["short_name"])

<a id='banking_account'></a>
### Banking - Account
<a href='#top'>top</a>

In [104]:
ui.click_side_menu("Banking","Accounts")

In [107]:
##press add new button
ui.click_add()

#input banking data 

for i in range(len(data["banking"])):  
    ui.enter_text((By.ID, "name"), data["banking"][i]["name"])
    ui.enter_text((By.ID, "number"), data["banking"][i]["number"])
    ui.enter_text((By.NAME, "opening_balance"), data["banking"][i]["openingbalance"])
    ui.enter_text((By.NAME, "bank_name"), data["banking"][i]["bankname"])
    ui.enter_text((By.NAME, "bank_phone"), data["banking"][i]["bankphone"])
    ui.enter_text((By.NAME, "bank_address"), data["banking"][i]["bankaddress"])
    ui.select_dropdown("Currency", data["banking"][i]["bankcurrency"])
    ui.select_toggle_button("Enabled", "Yes")
    ui.select_toggle_button("Default Account", data["banking"][i]["default"] )
    ui.click_save()
    instance.wait(2)
    locator = (By.XPATH, './/a[@class="btn btn-success btn-sm"]')
    instance.click_element(locator)
    



In [ ]:
instance.close()

<a id='banking_transfer'></a>
### Banking - Transfer
<a href='#top'>top</a>

In [108]:
ui.click_side_menu("Banking","Transfers")

In [110]:
for transfer in transfers:
    ui.click_add()
    instance.wait(2)
    ui.select_dropdown("From Account", transfer["faccount"])
    ui.select_dropdown("To Account", transfer["taccount"])
    ui.enter_text((By.NAME, "amount"), transfer["amount"])
    ui.select_date_picker("Date", transfer["date"])
    ui.enter_text((By.NAME, "description"), transfer["description"])
    ui.select_dropdown("Payment Method", transfer["payment_method"])
    ui.enter_text((By.ID, "reference"), transfer["reference"])
    ui.click_save()
    instance.wait(5)
    instance.click_element((By.XPATH, f'//a[@class="nav-link active"]//*[text()[contains(., "Transfers")]]'))
    instance.wait(2)

In [ ]:
instance.close()